# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'
#y0 = '2013'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

#north_pumas = ['0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,GASP,FULP,WATP,ELEP'

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
1000000 = 1.000000

FHINCP - income flag - 1: yes | https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

GRNTP - gross rent (monthly) (not 0: n/a,not paying rent, owned)
https://api.census.gov/data/2021/acs/acs1/pums/variables/GRNTP.json

GASP - gas cost (monthly) (not 3:N/a, included in rent/condo)
https://api.census.gov/data/2021/acs/acs1/pums/variables/GASP.json

FULP - fuel cost (yearly) (not 2:n/a, included in rent/condo)
https://api.census.gov/data/2021/acs/acs1/pums/variables/FULP.json

WATP - water cost (yearly) (not 2:n/a,included in rent/condo)
https://api.census.gov/data/2021/acs/acs1/pums/variables/WATP.json

ELEP - electricity cost (monthly) (2:N/A, included in rent/condo)
https://api.census.gov/data/2021/acs/acs1/pums/variables/ELEP.json

In [4]:
#AMI bands in 2021 by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [38]:
#Housing costs affordable to different AMI bands based on household size
aff_1p = {'30pct':461,'50pct':767,'80pct':1217,'100pct':1535}
aff_2p = {'30pct':527,'50pct':877,'80pct':1392,'100pct':1754}
aff_3p = {'30pct':609,'50pct':985,'80pct':1565,'100pct':1973}
aff_4p = {'30pct':735,'50pct':1096,'80pct':1738,'100pct':2192}
aff_5p = {'30pct':861,'50pct':1185,'80pct':1878,'100pct':2370}
aff_6p = {'30pct':987,'50pct':1272,'80pct':2017,'100pct':2545}
aff_7p = {'30pct':1113,'50pct':1360,'80pct':2156,'100pct':2720}
aff_8p = {'30pct':1239,'50pct':1447,'80pct':2295,'100pct':2894}

unit_afford = {'1':[0,461,767,1217,1535,1000000000],\
              '2':[0,527,877,1392,1535,1000000000],\
              '3':[0,609,985,1565,1973,1000000000],\
              '4':[0,735,1096,1738,2192,1000000000],\
              '5':[0,861,1185,1878,2370,1000000000],\
              '6':[0,987,1272,2017,2545,1000000000],\
              '7':[0,1113,1360,2156,2720,1000000000],\
              '8':[0,1239,1447,2295,2894,1000000000]}

inc_lbls = ['u30_ami','30_50_ami','50_80_ami','80_100_ami','o100_ami']

In [5]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [6]:
df = get.get_puma(sample,y1,data_cols)

In [7]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [8]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)
df['HINCP'] = df.ADJINC * df.HINCP

In [9]:
dff = df[~(df.HHSz.isna()) & (df.HINCP!=-60000)&(df.HINCP!=0)&(df.GRNTP!=0)].copy()

In [10]:
dff['gas'] = np.where(dff.GASP==3,0,dff.GASP)
dff['fuel'] = np.where(dff.FULP==2,0,(dff.FULP/12))
dff['water'] = np.where(dff.WATP==2,0,(dff.WATP/12))
dff['elec'] = np.where(dff.ELEP==2,0,dff.ELEP)

dff['utilities'] = dff.gas + dff.fuel + dff.water + dff.elec
dff['hou_cost'] = dff.GRNTP+dff.utilities
dff['aff_cost'] = np.where(dff.HINCP>=1,(dff.HINCP*0.333)/12,0)

In [11]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),'u30_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),'30_50_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),\
                   '50_80_ami',np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),\
                   '80_100_ami','o100_ami'))))

In [12]:
dff['cost_burdened'] = np.where(dff.hou_cost>dff.aff_cost,'burdened','not burdened')

In [40]:
#who is the unit affordable to based on the rent
dff['unit_aff'] = np.where(dff.HHSz=='1',pd.cut(dff['hou_cost'],bins=unit_afford['1'],labels=inc_lbls),\
                  np.where(dff.HHSz=='2',pd.cut(dff['hou_cost'],bins=unit_afford['2'],labels=inc_lbls),\
                  np.where(dff.HHSz=='3',pd.cut(dff['hou_cost'],bins=unit_afford['3'],labels=inc_lbls),\
                  np.where(dff.HHSz=='4',pd.cut(dff['hou_cost'],bins=unit_afford['4'],labels=inc_lbls),\
                  np.where(dff.HHSz=='5',pd.cut(dff['hou_cost'],bins=unit_afford['5'],labels=inc_lbls),\
                  np.where(dff.HHSz=='6',pd.cut(dff['hou_cost'],bins=unit_afford['6'],labels=inc_lbls),\
                  np.where(dff.HHSz=='7',pd.cut(dff['hou_cost'],bins=unit_afford['7'],labels=inc_lbls),\
                  np.where(dff.HHSz=='8',pd.cut(dff['hou_cost'],bins=unit_afford['8'],labels=inc_lbls),''))))))))

In [43]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,GASP,FULP,WATP,...,gas,fuel,water,elec,utilities,hou_cost,aff_cost,AMI_range,cost_burdened,unit_aff
3612,0400120,177353.6016,3.0,42.0,1.029928,1000000.0,2460.0,3.0,2.0,1200.0,...,0.0,0.0,100.000000,160.0,260.000000,2720.000000,4921.562444,o100_ami,not burdened,o100_ami
3637,0400119,55307.1336,7.0,172.0,1.029928,1000000.0,1405.0,3.0,2.0,1500.0,...,0.0,0.0,125.000000,480.0,605.000000,2010.000000,1534.772957,50_80_ami,burdened,50_80_ami
3639,0400116,70344.0824,2.0,177.0,1.029928,1000000.0,1808.0,3.0,2.0,100.0,...,0.0,0.0,8.333333,300.0,308.333333,2116.333333,1952.048287,o100_ami,burdened,o100_ami


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [67]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [51]:
drop_cols = ['HINCP','NP','ADJINC','ADJHSG','GRNTP','GASP','FULP','WATP',\
             'ELEP','gas', 'fuel', 'water', 'elec', 'utilities','hou_cost', 'aff_cost','HHSz']

In [64]:
table = dff.copy().drop(columns=drop_cols)

In [65]:
table = table.groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened']).sum().reset_index()

In [66]:
table.head()

,GEO_ID,AMI_range,unit_aff,cost_burdened,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
0,0400112,30_50_ami,50_80_ami,burdened,42.0,72.0,13.0,43.0,40.0,12.0,...,42.0,73.0,11.0,11.0,12.0,13.0,76.0,43.0,44.0,41.0
1,0400112,30_50_ami,80_100_ami,burdened,176.0,238.0,325.0,178.0,179.0,163.0,...,202.0,151.0,319.0,157.0,174.0,54.0,306.0,58.0,57.0,188.0
2,0400112,30_50_ami,o100_ami,burdened,1944.0,2176.0,2084.0,2279.0,2720.0,2157.0,...,2455.0,1519.0,1147.0,1705.0,1761.0,2256.0,1776.0,1594.0,2186.0,2131.0
3,0400112,50_80_ami,50_80_ami,not burdened,468.0,476.0,823.0,435.0,446.0,777.0,...,467.0,145.0,466.0,460.0,428.0,462.0,131.0,146.0,489.0,794.0
4,0400112,50_80_ami,80_100_ami,burdened,236.0,229.0,334.0,253.0,274.0,242.0,...,328.0,402.0,301.0,203.0,234.0,283.0,335.0,240.0,341.0,337.0


In [76]:
table_2 = table.copy().groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened']).sum().reset_index()
table_2 = make_est(table_2)
table_2 = table_2.drop(columns=repwts)

In [70]:
rent_ami = table.copy().drop(columns=['unit_aff','cost_burdened']).groupby(['GEO_ID','AMI_range']).sum().reset_index()
rent_ami = make_est(rent_ami)
rent_ami = rent_ami.drop(columns=repwts)

In [78]:
rent_aff = table.copy().drop(columns=['AMI_range','cost_burdened']).groupby(['GEO_ID','unit_aff']).sum().reset_index()
rent_aff = make_est(rent_aff)
rent_aff = rent_aff.drop(columns=repwts)

In [80]:
cost_burd = table.copy().drop(columns=['unit_aff','AMI_range']).groupby(['GEO_ID','cost_burdened']).sum().reset_index()
cost_burd = make_est(cost_burd)
cost_burd = cost_burd.drop(columns=repwts)

In [63]:
with pd.ExcelWriter(f'output/pums_rent_gap.xlsx') as writer:
    table_2.t
    rent_ami.to_excel(writer,sheet_name='hh_ami_lvl_renter')
    rent_aff.to_excel(writer,sheet_name='hh_afford_byAMI_renter')
    cost_burd.to_excel(writer,sheet_name='cost_burdened_byAMI_renter')